In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer
import regex as re

In [2]:
BH_tweets_before_pandemic = pd.read_csv('dane/BH_tweets_before_pandemic.csv', lineterminator='\n')

In [3]:
BH_tweets_before_pandemic.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video,near,geo,source,user_rt_id,user_rt,retweet_id,retweet_date,translate,trans_src,trans_dest
count,4.194200e+04,4.194200e+04,41942.0,4.194200e+04,41942.000000,41942.000000,41942.000000,41942.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.011567e+18,1.011450e+18,200.0,2.144457e+17,0.166301,0.738043,2.586858,0.299485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.856332e+16,1.925665e+16,0.0,3.784974e+17,5.611189,14.171729,149.405324,0.458038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,9.802449e+17,4.171556e+17,200.0,3.240000e+02,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,9.934337e+17,9.934106e+17,200.0,1.159675e+08,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.009675e+18,1.009642e+18,200.0,8.484296e+08,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.027687e+18,1.027664e+18,200.0,4.839383e+09,0.000000,0.000000,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.046548e+18,1.046548e+18,200.0,1.044284e+18,1092.000000,2350.000000,30265.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
BH_tweets_before_pandemic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41942 entries, 0 to 41941
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               41942 non-null  int64  
 1   conversation_id  41942 non-null  int64  
 2   created_at       41942 non-null  object 
 3   date             41942 non-null  object 
 4   time             41942 non-null  object 
 5   timezone         41942 non-null  int64  
 6   user_id          41942 non-null  int64  
 7   username         41942 non-null  object 
 8   name             41940 non-null  object 
 9   place            431 non-null    object 
 10  tweet            41942 non-null  object 
 11  language         41942 non-null  object 
 12  mentions         41942 non-null  object 
 13  urls             41942 non-null  object 
 14  photos           41942 non-null  object 
 15  replies_count    41942 non-null  int64  
 16  retweets_count   41942 non-null  int64  
 17  likes_count 

In [5]:
BH_tweets_before_pandemic.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1035675683450761217,1035675683450761217,2018-09-01 01:48:19 CEST,2018-09-01,01:48:19,200,27569109,albeesgirl,Staci Rullo,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1035673787356864512,1035673787356864512,2018-09-01 01:40:47 CEST,2018-09-01,01:40:47,200,3296463285,k_val81,Karen Valentine,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1035672623026397184,1035670992448675841,2018-09-01 01:36:09 CEST,2018-09-01,01:36:09,200,2322821784,stevet_140,Steve T,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'NastromKenneth', 'name': 'Ke...",NaN,NaN,NaN,NaN
3,1035669895151738880,1035669895151738880,2018-09-01 01:25:19 CEST,2018-09-01,01:25:19,200,332364947,simonsalhanyre,BHHS Simon & Salhany,"{'type': 'Point', 'coordinates': [41.193133, -...",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1035668062610116609,1035668062610116609,2018-09-01 01:18:02 CEST,2018-09-01,01:18:02,200,918188324426194944,stand_4_america,Free America Network,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
#Add date format and sorted by dates
BH_tweets_before_pandemic['date'] = pd.to_datetime(BH_tweets_before_pandemic['date'], format = '%Y-%m-%d')
BH_tweets_before_pandemic = BH_tweets_before_pandemic.sort_values(by='date')
BH_tweets_before_pandemic = BH_tweets_before_pandemic.reset_index(drop=True)
BH_tweets_before_pandemic

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,980244887508369408,980244887508369408,2018-04-01 02:46:07 CEST,2018-04-01,02:46:07,200,4839382570,canada_gaming,The Canadian Gamer,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,980503304261128193,980503304261128193,2018-04-01 19:52:59 CEST,2018-04-01,19:52:59,200,4499006729,greatmikehawes,Mike Hawes,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,980499856866164736,980499856866164736,2018-04-01 19:39:17 CEST,2018-04-01,19:39:17,200,971803047742656513,mosellsfmhomes,"Maureen Bartelt, REALTOR",NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,980494530804944898,980494530804944898,2018-04-01 19:18:07 CEST,2018-04-01,19:18:07,200,2503230883,_real__estate,Real Estate News,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,980492102260858880,980492102260858880,2018-04-01 19:08:28 CEST,2018-04-01,19:08:28,200,849762128,bhhspremier,Berkshire Hathaway HomeServices Premier Proper...,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41937,1046520067788279808,1046520067788279808,2018-10-01 00:00:02 CEST,2018-10-01,00:00:02,200,923066213156184064,bhhsmke,BHHSMetroRealty,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
41938,1046531227417759745,1046531227417759745,2018-10-01 00:44:22 CEST,2018-10-01,00:44:22,200,376898714,andreaszlavikte,Andrea Szlavik Team,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
41939,1046528180775874560,1046528180775874560,2018-10-01 00:32:16 CEST,2018-10-01,00:32:16,200,2768300434,carlton_amber,Amber Carlton,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
41940,1046528163038154752,1046528163038154752,2018-10-01 00:32:12 CEST,2018-10-01,00:32:12,200,827212847012773888,diamondrafargo,Diamond Realty Assoc,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [7]:
#Remove tweets writeen in other language than english
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic[BH_tweets_before_pandemic.language == 'en']
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic_clean.reset_index(drop=True)
BH_tweets_before_pandemic_clean.shape

(39582, 36)

In [8]:
#Remove columns without any data
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic_clean.dropna(thresh = 1, axis = 1)

In [9]:
#2360 records and 10 columns were removed
BH_tweets_before_pandemic_clean.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video
count,3.958200e+04,3.958200e+04,39582.0,3.958200e+04,39582.000000,39582.000000,39582.000000,39582.000000
mean,1.011660e+18,1.011547e+18,200.0,2.156098e+17,0.166919,0.747208,2.636400,0.301071
std,1.858508e+16,1.929199e+16,0.0,3.791561e+17,5.772845,14.570742,153.785907,0.458729
min,9.802449e+17,4.171556e+17,200.0,3.240000e+02,0.000000,0.000000,0.000000,0.000000
25%,9.934617e+17,9.934551e+17,200.0,1.142742e+08,0.000000,0.000000,0.000000,0.000000
50%,1.009756e+18,1.009722e+18,200.0,8.497621e+08,0.000000,0.000000,0.000000,0.000000
75%,1.027904e+18,1.027888e+18,200.0,4.852194e+09,0.000000,0.000000,0.000000,1.000000
max,1.046548e+18,1.046548e+18,200.0,1.044284e+18,1092.000000,2350.000000,30265.000000,1.000000


In [10]:
BH_tweets_before_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39582 entries, 0 to 39581
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               39582 non-null  int64         
 1   conversation_id  39582 non-null  int64         
 2   created_at       39582 non-null  object        
 3   date             39582 non-null  datetime64[ns]
 4   time             39582 non-null  object        
 5   timezone         39582 non-null  int64         
 6   user_id          39582 non-null  int64         
 7   username         39582 non-null  object        
 8   name             39580 non-null  object        
 9   place            420 non-null    object        
 10  tweet            39582 non-null  object        
 11  language         39582 non-null  object        
 12  mentions         39582 non-null  object        
 13  urls             39582 non-null  object        
 14  photos           39582 non-null  objec

In [11]:
BH_tweets_before_pandemic_clean.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
0,980244887508369408,980244887508369408,2018-04-01 02:46:07 CEST,2018-04-01,02:46:07,200,4839382570,canada_gaming,The Canadian Gamer,NaN,...,0,0,[],[],https://twitter.com/Canada_Gaming/status/98024...,False,NaN,1,https://pbs.twimg.com/media/DZqHh3bVwAEPS4l.jpg,[]
1,980503304261128193,980503304261128193,2018-04-01 19:52:59 CEST,2018-04-01,19:52:59,200,4499006729,greatmikehawes,Mike Hawes,NaN,...,0,0,[],[],https://twitter.com/GreatMikeHawes/status/9805...,False,NaN,1,https://pbs.twimg.com/media/DZtyjtBVAAAO85C.jpg,[]
2,980499856866164736,980499856866164736,2018-04-01 19:39:17 CEST,2018-04-01,19:39:17,200,971803047742656513,mosellsfmhomes,"Maureen Bartelt, REALTOR",NaN,...,0,0,[],[],https://twitter.com/MoSellsFMHomes/status/9804...,False,NaN,1,https://pbs.twimg.com/media/DZtvbCqUMAA9p09.jpg,[]
3,980494530804944898,980494530804944898,2018-04-01 19:18:07 CEST,2018-04-01,19:18:07,200,2503230883,_real__estate,Real Estate News,NaN,...,0,0,['homeservices'],[],https://twitter.com/_Real__Estate/status/98049...,False,NaN,1,https://pbs.twimg.com/media/DZtqlKQWkAoSJT5.jpg,[]
4,980492102260858880,980492102260858880,2018-04-01 19:08:28 CEST,2018-04-01,19:08:28,200,849762128,bhhspremier,Berkshire Hathaway HomeServices Premier Proper...,NaN,...,0,0,[],[],https://twitter.com/bhhspremier/status/9804921...,False,NaN,1,https://pbs.twimg.com/media/DZtoXr-UQAA_ORM.jpg,[]


In [12]:
#Check values of column place
BH_tweets_before_pandemic_clean['place'].value_counts()

{'type': 'Point', 'coordinates': [33.9032486, -84.46694957]}     30
{'type': 'Point', 'coordinates': [36.067886, -115.272926]}       14
{'type': 'Point', 'coordinates': [32.2125, -80.8973]}             9
{'type': 'Point', 'coordinates': [46.8191027, -96.8532566]}       8
{'type': 'Point', 'coordinates': [40.7399954, -74.0301392]}       7
                                                                 ..
{'type': 'Point', 'coordinates': [41.114826, -81.57183]}          1
{'type': 'Point', 'coordinates': [41.060802, -81.510901]}         1
{'type': 'Point', 'coordinates': [40.7007, -99.0811]}             1
{'type': 'Point', 'coordinates': [38.59669502, -77.27554564]}     1
{'type': 'Point', 'coordinates': [36.0292, -115.025]}             1
Name: place, Length: 279, dtype: int64

In [13]:
#Check values of column quote_url
BH_tweets_before_pandemic_clean['quote_url'].value_counts()

0                                                                35
https://twitter.com/FoxBusiness/status/1034087310084722690        8
https://twitter.com/coindesk/status/990969761080250371            7
https://twitter.com/statnews/status/1009434526618128385           5
https://twitter.com/BloombergQuint/status/992777804541583363      4
                                                                 ..
https://twitter.com/solarcollab/status/996435133400952832         1
https://twitter.com/amitranjan/status/993904412698578945          1
https://twitter.com/forbes/status/999631354957529088              1
https://twitter.com/Sandro_power/status/1022573764742180864       1
https://twitter.com/HeinzKetchup_US/status/988793147126886402     1
Name: quote_url, Length: 525, dtype: int64

In [14]:
#Counted quantity of links for each tweet
BH_tweets_before_pandemic_clean['quote_url'] = BH_tweets_before_pandemic_clean['quote_url'].str.count('https://')

#Replayce NaN by 0
BH_tweets_before_pandemic_clean.fillna({'quote_url':0}, inplace=True)

#Changed data type from float64 to int64
BH_tweets_before_pandemic_clean['quote_url'] = BH_tweets_before_pandemic_clean['quote_url'].astype(np.int64)

In [15]:
#Replayce NaN to [] in thumbnail column
BH_tweets_before_pandemic_clean.fillna({'thumbnail':'[]'}, inplace=True)

In [16]:
#There is no NaN except column place which will be removed due to low importance
BH_tweets_before_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39582 entries, 0 to 39581
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               39582 non-null  int64         
 1   conversation_id  39582 non-null  int64         
 2   created_at       39582 non-null  object        
 3   date             39582 non-null  datetime64[ns]
 4   time             39582 non-null  object        
 5   timezone         39582 non-null  int64         
 6   user_id          39582 non-null  int64         
 7   username         39582 non-null  object        
 8   name             39580 non-null  object        
 9   place            420 non-null    object        
 10  tweet            39582 non-null  object        
 11  language         39582 non-null  object        
 12  mentions         39582 non-null  object        
 13  urls             39582 non-null  object        
 14  photos           39582 non-null  objec

In [17]:
#Check duplicates
BH_tweets_before_pandemic_clean[BH_tweets_before_pandemic_clean.duplicated(subset = 'id')]

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to


In [18]:
#Column retweet consist of only False values
BH_tweets_before_pandemic_clean['retweet'].value_counts()

False    39582
Name: retweet, dtype: int64

In [19]:
#Removed irrelevant columns
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic_clean.drop(['conversation_id', 
                                                                        'created_at', 'time', 'timezone', 
                                                                        'language', 'retweet', 'link','place'], 
                                                                       axis = 1)

In [20]:
#Counted number of elements stored as lists in columns
columns_lists = ['mentions', 'urls', 'photos', 'hashtags', 'cashtags', 'thumbnail']
for column in columns_lists:
    empty_list = BH_tweets_before_pandemic_clean[column] == '[]'
    BH_tweets_before_pandemic_clean[column] = BH_tweets_before_pandemic_clean[column].str[1:-1].str.split(',')
    BH_tweets_before_pandemic_clean[column] = BH_tweets_before_pandemic_clean[column].str.len()
    BH_tweets_before_pandemic_clean.loc[empty_list, column] = 0

In [21]:
#Counted number of elements in dictionaries stored in reply_to column
BH_tweets_before_pandemic_clean['reply_to'] = BH_tweets_before_pandemic_clean['reply_to'].str.count('{')

#Changed data type from float64 to int64
BH_tweets_before_pandemic_clean['reply_to'] = BH_tweets_before_pandemic_clean['reply_to'].astype(np.int64)

In [22]:
#Removed hastags, mentions, url and special signs from tweets
for i in range(BH_tweets_before_pandemic_clean.shape[0]):
    BH_tweets_before_pandemic_clean.loc[i, 'tweet'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",
                            BH_tweets_before_pandemic_clean.loc[i, 'tweet']).split())

In [23]:
#Last day 2018-10-01 has only 10 tweets, it will be removed
BH_tweets_before_pandemic_clean['date'].value_counts()

2018-05-05    2032
2018-05-04    1691
2018-08-27    1160
2018-05-07    1093
2018-06-20     975
              ... 
2018-07-29      82
2018-08-19      81
2018-04-01      80
2018-08-26      62
2018-10-01      10
Name: date, Length: 184, dtype: int64

In [24]:
BH_tweets_before_pandemic_clean = BH_tweets_before_pandemic_clean[BH_tweets_before_pandemic_clean['date'] != '2018-10-01']
BH_tweets_before_pandemic_clean['date'].value_counts()

2018-05-05    2032
2018-05-04    1691
2018-08-27    1160
2018-05-07    1093
2018-06-20     975
              ... 
2018-09-16      86
2018-07-29      82
2018-08-19      81
2018-04-01      80
2018-08-26      62
Name: date, Length: 183, dtype: int64

In [25]:
#Checked if there are tweets that contains only white spaces, numbers or special signs
BH_tweets_before_pandemic_clean[~BH_tweets_before_pandemic_clean["tweet"].str.contains('[A-Za-z]')]

,id,date,user_id,username,name,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,quote_url,video,thumbnail,reply_to


In [26]:
BH_tweets_before_pandemic_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39572 entries, 0 to 39571
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              39572 non-null  int64         
 1   date            39572 non-null  datetime64[ns]
 2   user_id         39572 non-null  int64         
 3   username        39572 non-null  object        
 4   name            39570 non-null  object        
 5   tweet           39572 non-null  object        
 6   mentions        39572 non-null  int64         
 7   urls            39572 non-null  int64         
 8   photos          39572 non-null  int64         
 9   replies_count   39572 non-null  int64         
 10  retweets_count  39572 non-null  int64         
 11  likes_count     39572 non-null  int64         
 12  hashtags        39572 non-null  int64         
 13  cashtags        39572 non-null  int64         
 14  quote_url       39572 non-null  int64         
 15  vi

In [27]:
#Save cleaned data in csv format
BH_tweets_before_pandemic_clean.to_csv('dane/BH_tweets_before_pandemic_clean.csv', index=False)

In [28]:
#Create dataframe for textanalysis for each day
BH_tweets_before_pandemic_textanalysis = pd.date_range(BH_tweets_before_pandemic_clean['date'].min(),
                                                       BH_tweets_before_pandemic_clean['date'].max(),freq='d')
BH_tweets_before_pandemic_textanalysis = pd.DataFrame(BH_tweets_before_pandemic_textanalysis, columns = ['date'])
column_names = ['Short negative factual tweets', 'Short negative personal tweets', 'Short positive factual tweets',
                'Short positive personal tweets', 'Short neutral factual tweets', 'Short neutral personal tweets',
                'Long negative factual tweets', 'Long negative personal tweets', 'Long positive factual tweets',
                'Long positive personal tweets', 'Long neutral factual tweets', 'Long neutral personal tweets',
                'Total amount of tweets']
for column in column_names:
    BH_tweets_before_pandemic_textanalysis[column] = 0
BH_tweets_before_pandemic_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2018-04-01,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018-04-02,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018-04-03,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2018-04-04,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2018-04-05,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,2018-09-26,0,0,0,0,0,0,0,0,0,0,0,0,0
179,2018-09-27,0,0,0,0,0,0,0,0,0,0,0,0,0
180,2018-09-28,0,0,0,0,0,0,0,0,0,0,0,0,0
181,2018-09-29,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
#Analysis of opinion and number of words using the textblob package
for i in range(BH_tweets_before_pandemic_clean.shape[0]):
    blob = TextBlob(BH_tweets_before_pandemic_clean.loc[i, 'tweet'], analyzer=PatternAnalyzer())
    if len(blob.words) < 30:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative factual tweets'] += 1
            else:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive factual tweets'] += 1
            else:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral factual tweets'] += 1
            else:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral personal tweets'] += 1
    else:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative factual tweets'] += 1
            else:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive factual tweets'] += 1
            else:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral factual tweets'] += 1
            else:
                BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral personal tweets'] += 1
    BH_tweets_before_pandemic_textanalysis.loc[BH_tweets_before_pandemic_textanalysis[BH_tweets_before_pandemic_textanalysis['date'] == BH_tweets_before_pandemic_clean.loc[i, 'date']].index.values[0], 
                                                               'Total amount of tweets'] += 1

In [30]:
BH_tweets_before_pandemic_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2018-04-01,3,2,22,5,36,1,0,1,4,2,1,3,80
1,2018-04-02,3,1,16,15,47,3,2,1,6,6,3,2,105
2,2018-04-03,5,1,29,24,70,3,4,2,9,11,5,1,164
3,2018-04-04,6,2,22,24,58,0,4,0,5,3,7,1,132
4,2018-04-05,14,1,47,24,58,2,8,1,12,12,2,2,183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,2018-09-26,39,3,44,80,56,2,1,1,12,14,0,1,253
179,2018-09-27,12,2,54,43,81,5,5,1,27,12,2,1,245
180,2018-09-28,12,3,33,37,67,4,0,0,12,14,3,1,186
181,2018-09-29,5,1,21,18,32,2,1,1,6,10,5,0,102


In [31]:
BH_tweets_before_pandemic_textanalysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            183 non-null    datetime64[ns]
 1   Short negative factual tweets   183 non-null    int64         
 2   Short negative personal tweets  183 non-null    int64         
 3   Short positive factual tweets   183 non-null    int64         
 4   Short positive personal tweets  183 non-null    int64         
 5   Short neutral factual tweets    183 non-null    int64         
 6   Short neutral personal tweets   183 non-null    int64         
 7   Long negative factual tweets    183 non-null    int64         
 8   Long negative personal tweets   183 non-null    int64         
 9   Long positive factual tweets    183 non-null    int64         
 10  Long positive personal tweets   183 non-null    int64         
 11  Long n

In [32]:
#Save text analysis data in csv format
BH_tweets_before_pandemic_textanalysis.to_csv('dane/BH_tweets_before_pandemic_textanalysis.csv', index=False)